# Data Load

In [74]:
import pandas as pd
import pprint as pp
data = pd.read_excel('data/train_set/data.xlsx', header = 1)

###### data 확인
### 1. NAN 처리

In [73]:
data.loc[data['방송일시'] =='2019-03-03 00:00:00']

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
5922,2019-03-03,20.0,100271,200889,헤스티지 그레이스 양가죽 코트,의류,199000,17737000.0
5923,2019-03-03,NaN,100270,200892,헤스티지 마론 양가죽 남성집업재킷,의류,199000,15828000.0


In [75]:
data.dtypes

방송일시     datetime64[ns]
노출(분)           float64
마더코드              int64
상품코드              int64
상품명              object
상품군              object
판매단가              int64
취급액             float64
dtype: object

## 1. nan 확인
- 노출(분) 항목은 약 43%가 nan
- 취급액은 약 2.4% 가 nan

In [76]:
data.isnull().sum()*100/len(data)

방송일시      0.000000
노출(분)    43.812159
마더코드      0.000000
상품코드      0.000000
상품명       0.000000
상품군       0.000000
판매단가      0.000000
취급액       2.445900
dtype: float64

- 취급액이 0 인 항목 확인
- 937개가 있으며, 이것들은 **예측 상품 중 판매가 0인 프로그램 실적은 예측에서 제외함** 이라고 데이터 설명서에 되어있음
- 취급액이 NaN인 것은 보험, 무형 상품이고, 삭제해도 된다.

In [77]:
data.loc[data['취급액'].isnull()].head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
2318,2019-01-25 19:00:00,60.0,100507,201567,TCOM_처브치아보험,무형,0,NaN
2845,2019-01-31 18:00:00,60.0,100507,201567,TCOM_처브치아보험,무형,0,NaN
3332,2019-02-05 18:00:00,60.0,100507,201567,TCOM_처브치아보험,무형,0,NaN
6570,2019-03-09 00:00:00,20.0,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
6571,2019-03-09 00:20:00,20.0,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN


In [78]:
print(data.loc[data['취급액'].isnull()]['상품군'].unique())

['무형']


### 1.1 취급액이 NaN인 항목 삭제

In [91]:
data_clean1 = data.dropna(subset = ['취급액'])
data_clean1

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
38301,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
38302,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


### 1.3 노출시간이 nan 인것 확인
    - 동일한 시간대에 서로다른 상품코드의 제품을 팔때에는 노출(분)이 nan으로 기록이 된다.
    - 마더코드가 다를 수 있다.
    - 동시간대에 판매한 것은 노출(분)이 동일하다고 가정한다.(채널이 한개 뿐)
    - ex : 여성옷, 남성옷, 무이자, 일시불

In [81]:
# 노출(분)이 nan인 데이터를 찾아서 가져온 뒤, 노출(분) 항목을 지워준다
nans = data.loc[(data.loc[data['노출(분)'].isnull()].index)].drop('노출(분)', axis = 1)
nans.head(3)

,방송일시,마더코드,상품코드,상품명,상품군,판매단가,취급액
1,2019-01-01 06:00:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
3,2019-01-01 06:20:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
5,2019-01-01 06:40:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0


In [82]:
# 마더코드 - 방송일시 - 노출(분) 데이터를 만든다. 같은 마더코드, 방송일시를 가진 상품은 노출(분)이 동일하다.
code_bun = data[['노출(분)','방송일시']].dropna().drop_duplicates().reset_index(drop = True)
code_bun.loc[code_bun['방송일시'] == '2019-03-12 00:20:00']

,노출(분),방송일시
4049,20.0,2019-03-12 00:20:00


In [83]:
nans

,방송일시,마더코드,상품코드,상품명,상품군,판매단가,취급액
1,2019-01-01 06:00:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
3,2019-01-01 06:20:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
5,2019-01-01 06:40:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0
26,2019-01-01 14:00:00,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,20841000.0
28,2019-01-01 14:30:00,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,47294000.0
...,...,...,...,...,...,...,...
38298,2019-12-31 23:40:00,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0
38299,2019-12-31 23:40:00,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
38301,2020-01-01 00:00:00,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
38302,2020-01-01 00:00:00,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


In [84]:
nans_merged = pd.merge(nans,code_bun, how = 'left', on ='방송일시')
nans_merged

,방송일시,마더코드,상품코드,상품명,상품군,판매단가,취급액,노출(분)
0,2019-01-01 06:00:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,20.0
1,2019-01-01 06:20:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,20.0
2,2019-01-01 06:40:00,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0,20.0
3,2019-01-01 14:00:00,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,20841000.0,30.0
4,2019-01-01 14:30:00,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,47294000.0,30.0
...,...,...,...,...,...,...,...,...
16779,2019-12-31 23:40:00,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,20.0
16780,2019-12-31 23:40:00,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,20.0
16781,2020-01-01 00:00:00,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,20.0
16782,2020-01-01 00:00:00,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,20.0


In [97]:
data_remove_nan = pd.concat([data_clean1.dropna(subset=['노출(분)']), nans_merged])
data_remove_nan.sort_values(by = ['방송일시','마더코드','상품코드']).reset_index(drop = True)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
37369,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0
37370,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0


In [99]:
# nan 제거된 버젼 저장
data_remove_nan.to_csv('data/train_set/data_remove_nan.csv', index = False)